# Create Knowledge Graph

The following notebook creates a small Knowledge Graph around Insurance data. This fake/dummy data reprents the following: 
- **Organizations**: The policies are provided by different insurance organizations.
- **Insurances**: The Insurance "Products" that are provided by the insurance organizations.
- **Clients**: Each insurance may have different clients that bought this product.\

In the following notebook this fake data is created as an example and loaded to Neo4j. The loading is done using the [Python Driver](https://neo4j.com/docs/api/python-driver/current/) that enables querying from a Python script

In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from IPython.display import clear_output

## Get Credentials

In [2]:
if os.path.exists('credentials.env'):
    load_dotenv('credentials.env', override=True)

    # Neo4j
    uri = os.getenv('NEO4J_URI')
    username = os.getenv('NEO4J_USERNAME')
    password = os.getenv('NEO4J_PASSWORD')
    database = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
else:
    print("File 'credentials.env' not found.")

## Setup Connection to Database

Setup connection to the database with the Python Driver

In [3]:
class App:
    def __init__(self, uri, user, password, database=None):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database=database)
        self.database = database

    def close(self):
        self.driver.close()

    def query(self, query):
        return self.driver.execute_query(query)

    def query_params(self, query, parameters):
        return self.driver.execute_query(query, parameters_=parameters)

    def count_nodes_in_db(self):
        query = "MATCH (n) RETURN COUNT(n)"
        result = self.query(query)
        (key, value) = result.records[0].items()[0]
        return value

    def remove_nodes_relationships(self):
        query ="""
            CALL apoc.periodic.iterate(
                "MATCH (c) RETURN c",
                "WITH c DETACH DELETE c",
                {batchSize: 1000}
            )
        """
        result = self.query(query)

    def remove_all_constraints(self):
        query ="""
            CALL apoc.schema.assert({}, {})
        """
        result = self.query(query)

In [4]:
app = App(uri, username, password, database)

In [5]:
app.count_nodes_in_db()

977

## Load Policies from Neo4j

In [6]:
query = """
    MATCH (p:Policy)
    RETURN p.id AS policy_id, p.file_name AS file_name
"""

In [7]:
result = app.query(query)

In [8]:
data = [record.values() for record in result.records]
df = pd.DataFrame(data = data, columns = ['policy_id', 'file_name'])
df

,policy_id,file_name
0,0,NN_zorg_vrij_basic_2024.pdf


### Create Organization Nodes

The documents come from official websites of organizations. Now we make some nodes per organization.

In [9]:
data = [
    ['FBTO', 0],
    ['Nationale Nederlanden', 1],
    ['Zilveren Kruis', 2]
]

In [10]:
organization_df= pd.DataFrame(
    data = data,
    columns = ['name', 'id']
)

In [11]:
organization_df

,name,id
0,FBTO,0
1,Nationale Nederlanden,1
2,Zilveren Kruis,2


### Create Insurance Nodes

The insurances are described in the policies. Now create some insurance nodes from these filenames.

In [12]:
insurances_df = df[['file_name', 'policy_id']].drop_duplicates()

In [13]:
insurances_df['insurance_name'] = insurances_df['file_name'].apply(lambda x: x.replace('.pdf', '').replace('_', ' '))
insurances_df['id'] = insurances_df['policy_id']
insurances_df.drop(['policy_id', 'file_name'], axis=1, inplace=True)

In [14]:
insurances_df

,insurance_name,id
0,NN zorg vrij basic 2024,0


### Create Client Nodes

Create some fake client data

In [15]:
data = [
    ['Erik', 0],
    ['Thorsten', 1],
    ['Hylke', 2],
    ['Marco', 3],
    ['Kees', 4],
    ['Niels', 5]
]

In [16]:
clients_df = pd.DataFrame(
    data = data,
    columns = ['name', 'id']
)

In [17]:
clients_df

,name,id
0,Erik,0
1,Thorsten,1
2,Hylke,2
3,Marco,3
4,Kees,4
5,Niels,5


### Create PROVIDES  Relationship

Link the data from organization to insurance by a provides DataFrame

In [18]:
data = [
    [0, 'voorwaarden FBTO zorgverzekering 2024'],
    [1, 'NN zorg basis 2024'],
    [2, 'zilveren kruis basis 2024'],
    [1, 'NN zorg vrij basic 2024'],
    [1, 'NN zorg top 2024']
]

In [19]:
provides_df = pd.DataFrame(
    data = data,
    columns = ['organization_id', 'insurance_name']
)

In [20]:
provides_df

,organization_id,insurance_name
0,0,voorwaarden FBTO zorgverzekering 2024
1,1,NN zorg basis 2024
2,2,zilveren kruis basis 2024
3,1,NN zorg vrij basic 2024
4,1,NN zorg top 2024


### Create INSURES Relationship

Link the data from insurance to client by an INSURES DataFrame

In [21]:
data = [
    ['Erik', 'voorwaarden FBTO zorgverzekering 2024'],
    ['Thorsten', 'NN zorg top 2024'],
    ['Hylke', 'zilveren kruis basis 2024'],
    ['Marco', 'NN zorg vrij basic 2024'],
    ['Kees', 'NN zorg basis 2024'],
    ['Niels', 'NN zorg basis 2024']
]

In [22]:
insures_df = pd.DataFrame(
    data = data,
    columns = ['client_name', 'insurance_name']
)

In [23]:
insures_df

,client_name,insurance_name
0,Erik,voorwaarden FBTO zorgverzekering 2024
1,Thorsten,NN zorg top 2024
2,Hylke,zilveren kruis basis 2024
3,Marco,NN zorg vrij basic 2024
4,Kees,NN zorg basis 2024
5,Niels,NN zorg basis 2024


### Create HAS_POLICY Relationship

Link the data from insurance to policy by a HAS_POLICY DataFrame

In [24]:
data = [
    [0, 0],
    [1, 1],
    [2, 2],
    [3, 3],
    [4, 4],
]

In [25]:
has_policy_df = pd.DataFrame(
    data = data,
    columns = ['insurance_id', 'policy_id']
)

In [26]:
has_policy_df

,insurance_id,policy_id
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


### Create some constraints

In [27]:
app.query("CREATE CONSTRAINT unique_organization IF NOT EXISTS FOR (o:Organization) REQUIRE o.id IS UNIQUE")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x11f9e2b80>, keys=[])

In [28]:
app.query("CREATE CONSTRAINT unique_insurance IF NOT EXISTS FOR (i:Insurance) REQUIRE i.id IS UNIQUE")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x108cb5a90>, keys=[])

In [29]:
app.query("CREATE CONSTRAINT unique_client IF NOT EXISTS FOR (c:Client) REQUIRE c.id IS UNIQUE")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x11f9bbb50>, keys=[])

# Load Nodes

### Load Organizations

In [30]:
organization_df

,name,id
0,FBTO,0
1,Nationale Nederlanden,1
2,Zilveren Kruis,2


In [31]:
merge_organizations_query = """
    MERGE(mergedOrg:Organization {id: $id})
        ON CREATE SET
            mergedOrg.name = $name
    RETURN mergedOrg
"""

In [32]:
for index, row in organization_df.iterrows():
    clear_output(wait=True)
    d = {
        'id': row['id'],
        'name': row['name']
    }
    app.query_params(merge_organizations_query, d)
    print("Progress: ", np.round((index+1)/organization_df.shape[0]*100,2), "%")

Progress:  100.0 %


### Load Insurances

In [33]:
insurances_df

,insurance_name,id
0,NN zorg vrij basic 2024,0


In [34]:
merge_insurance_query = """
    MERGE(mergedIns:Insurance {id: $id})
        ON CREATE SET
            mergedIns.name = $name
    RETURN mergedIns
"""

In [35]:
for index, row in insurances_df.iterrows():
    clear_output(wait=True)
    d = {
        'id': row['id'],
        'name': row['insurance_name'],
    }
    app.query_params(merge_insurance_query, d)
    print("Progress: ", np.round((index+1)/insurances_df.shape[0]*100,2), "%")

Progress:  100.0 %


### Load Clients

In [36]:
clients_df

,name,id
0,Erik,0
1,Thorsten,1
2,Hylke,2
3,Marco,3
4,Kees,4
5,Niels,5


In [37]:
merge_client_query = """
    MERGE(mergedClient:Client {id: $id})
        ON CREATE SET
            mergedClient.name = $name
    RETURN mergedClient
"""

In [38]:
for index, row in clients_df.iterrows():
    clear_output(wait=True)
    d = {
        'id': row['id'],
        'name': row['name'],
    }
    app.query_params(merge_client_query, d)
    print("Progress: ", np.round((index+1)/clients_df.shape[0]*100,2), "%")

Progress:  100.0 %


## Load Relationships

### Load provides

In [39]:
provides_df

,organization_id,insurance_name
0,0,voorwaarden FBTO zorgverzekering 2024
1,1,NN zorg basis 2024
2,2,zilveren kruis basis 2024
3,1,NN zorg vrij basic 2024
4,1,NN zorg top 2024


In [40]:
merge_provides_query = """
    MATCH
        (org:Organization {id: $org_id}),
        (ins:Insurance {name: $ins_name})
    MERGE (org)-[r:PROVIDES]->(ins)
    RETURN org.name, type(r), ins.name
"""

In [41]:
for index, row in provides_df.iterrows():
    clear_output(wait=True)
    d = {
        'org_id': row['organization_id'],
        'ins_name': row['insurance_name']
    }
    app.query_params(merge_provides_query, d)
    print("Progress: ", np.round(((index+1)/provides_df.shape[0])*100,2), "%")

Progress:  100.0 %


### Load INSURES

In [42]:
insures_df

,client_name,insurance_name
0,Erik,voorwaarden FBTO zorgverzekering 2024
1,Thorsten,NN zorg top 2024
2,Hylke,zilveren kruis basis 2024
3,Marco,NN zorg vrij basic 2024
4,Kees,NN zorg basis 2024
5,Niels,NN zorg basis 2024


In [43]:
merge_insures_query = """
    MATCH
        (ins:Insurance {name: $ins_name}),
        (client:Client {name: $client_name})
    MERGE (ins)-[r:INSURES]->(client)
    RETURN ins.name, type(r), client.name
"""

In [44]:
for index, row in insures_df.iterrows():
    clear_output(wait=True)
    d = {
        'ins_name': row['insurance_name'],
        'client_name': row['client_name']
    }
    app.query_params(merge_insures_query, d)
    print("Progress: ", np.round(((index+1)/insures_df.shape[0])*100,2), "%")

Progress:  100.0 %


### Load HAS_POLICY

In [45]:
has_policy_df

,insurance_id,policy_id
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [46]:
merge_insures_query = """
    MATCH
        (ins:Insurance {id: $ins_id}),
        (policy:Policy {id: $policy_id})
    MERGE (ins)-[r:HAS_POLICY]->(policy)
    RETURN ins.name, type(r), policy.file_name
"""

In [47]:
for index, row in has_policy_df.iterrows():
    clear_output(wait=True)
    d = {
        'ins_id': row['insurance_id'],
        'policy_id': row['policy_id']
    }
    app.query_params(merge_insures_query, d)
    print("Progress: ", np.round(((index+1)/has_policy_df.shape[0])*100,2), "%")

Progress:  100.0 %
